In [34]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import re

In [35]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
68375,"Hey, be careful with that!","Hé, faites attention avec ça !",CC-BY 2.0 (France) Attribution: tatoeba.org #2...
122509,I'm afraid we don't have any left.,J'ai peur que nous n'en ayons plus.,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
110872,I lived in Sanda City last year.,L'année dernière je vivais à Sanda City.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
150764,We've been unable to determine the cause.,Nous avons été incapables d'en déterminer la c...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
7378,Did I hurt you?,Vous ai-je blessé ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [36]:
lines = lines[['eng', 'fra']][:33000] # 3만3000개 샘플 사용
lines.sample(5)

,eng,fra
23065,Can you believe it?,Arrives-tu à le croire ?
32551,Tom works in Boston.,Tom travaille à Boston.
15146,I really mean it.,Je suis sérieuse.
17705,You need to stop.,Il faut que vous arrêtiez.
8621,Is French easy?,Le français est-il facile ?


# 소문자로 변형 및 특수문자 제거

In [37]:
lines['eng'] = lines['eng'].str.lower()
lines['fra'] = lines['fra'].str.lower()
lines['eng'] = lines['eng'].str.replace('\'s', ' is') # 's 를 is 로 변경
lines['eng'] = lines['eng'].str.replace('\'re', ' are') # 're --> are
lines['eng'] = lines['eng'].str.replace('\'m', ' am') 
print(lines.loc[[16439]])
print(lines.loc[[25128]])
print(lines.loc[[22807]])
print(lines.loc[[31305]])
                                        
lines['eng'] = lines['eng'].str.replace('\W', ' ') #영어, 프랑스어 특수문자 모두 제거
lines['fra'] = lines['fra'].str.replace('\W', ' ') 
#lines['eng'] = lines['eng'].str[:-1] + ' ' +lines['eng'].str[-1:]

print(lines)

                      eng                 fra
16439  that is fantastic.  c'est fantastique.
                        eng                           fra
25128  i am still in shock.  je suis encore sous le choc.
                       eng                        fra
22807  you are very brave.  vous êtes fort courageux.
                         eng                     fra
31305  now let is celebrate.  maintenant, fêtons ça.
                        eng                              fra
0                       go                              va  
1                       hi                           salut  
2                       hi                            salut 
3                      run                           cours  
4                      run                          courez  
...                     ...                              ...
32995  what was their goal             quel était leur but  
32996  what were you doing   qu étais tu en train de faire  
32997  what would tom need   

In [38]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
22327,why are you alone,\t pourquoi êtes vous seules \n
18000,advance two steps,\t avance de deux pas \n
30082,i said it as a joke,\t je l ai dit comme une blague \n
8934,no one can say,\t personne ne peut le dire \n
25797,now is your chance,\t c est l occasion ou jamais \n


In [39]:
eng_tokenizer = Tokenizer(char_level=True) # 문자 단위로 Tokenizer를 생성합니다.
eng_tokenizer.fit_on_texts(lines.eng) # 33000개의 행을 가진 eng의 각 행에 토큰화를 수행

input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[17, 5, 1], [9, 3, 1], [9, 3, 1]]

In [40]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 33000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[10, 1, 18, 4, 1, 1, 1, 11],
 [10, 1, 3, 4, 13, 8, 5, 1, 1, 1, 11],
 [10, 1, 3, 4, 13, 8, 5, 1, 1, 11]]

In [41]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 39
프랑스어 단어장의 크기 : 53


In [42]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 61


In [43]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 39
프랑스어 단어장의 크기 : 53
영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 61


In [44]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [45]:
print(decoder_input[:3])
print(decoder_target[:3])

[[10, 1, 18, 4, 1, 1, 1], [10, 1, 3, 4, 13, 8, 5, 1, 1, 1], [10, 1, 3, 4, 13, 8, 5, 1, 1]]
[[1, 18, 4, 1, 1, 1, 11], [1, 3, 4, 13, 8, 5, 1, 1, 1, 11], [1, 3, 4, 13, 8, 5, 1, 1, 11]]


In [46]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 22)
프랑스어 입력데이터의 크기(shape) : (33000, 61)
프랑스어 출력데이터의 크기(shape) : (33000, 61)


In [47]:
print(encoder_input[0])

[17  5  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [48]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 22, 39)
프랑스어 입력데이터의 크기(shape) : (33000, 61, 53)
프랑스어 출력데이터의 크기(shape) : (33000, 61, 53)


In [49]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (33000, 22, 39)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 61, 53)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 61, 53)


In [50]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

print('⏳')

⏳


from tensorflow.keras.layers import Input, Embedding, Masking

#인코더에서 사용할 임베딩 층
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(33000, 16)(encoder_inputs)
encoder_lstm = LSTM(units = 256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
print(encoder_outputs)

#디코더에서 사용할 임베딩 층

decoder_inputs = Input(shape=(None,))
dec_emb =  Embedding(33000, 16)(decoder_inputs)
decoder_lstm = LSTM(units = 256, return_state=True)
decoder_outputs, state_h, state_c = decoder_lstm(dec_emb)

In [51]:
#입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
#hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
#디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
#hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]
print(encoder_outputs)

Tensor("lstm_8/PartitionedCall:0", shape=(None, 256), dtype=float32)


In [52]:
#입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
#hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
#decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

In [53]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [54]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit([encoder_input_train, decoder_input_train], decoder_target_train,
          batch_size=1024,
          epochs=20,
          validation_split = 0.2)

Epoch 1/20
24/24 [==============================] - 16s 662ms/step - loss: 1.8213 - val_loss: 1.5508
Epoch 2/20
24/24 [==============================] - 15s 643ms/step - loss: 1.2738 - val_loss: 1.3183
Epoch 3/20
24/24 [==============================] - 15s 640ms/step - loss: 1.1704 - val_loss: 1.2871
Epoch 4/20
24/24 [==============================] - 15s 628ms/step - loss: 1.0948 - val_loss: 1.2753
Epoch 5/20
24/24 [==============================] - 15s 630ms/step - loss: 1.0393 - val_loss: 1.1610
Epoch 6/20
24/24 [==============================] - 15s 629ms/step - loss: 0.9778 - val_loss: 1.0992
Epoch 7/20
24/24 [==============================] - 15s 630ms/step - loss: 0.9139 - val_loss: 1.0445
Epoch 8/20
24/24 [==============================] - 15s 631ms/step - loss: 0.8624 - val_loss: 0.9881
Epoch 9/20
24/24 [==============================] - 15s 630ms/step - loss: 0.8187 - val_loss: 0.9429
Epoch 10/20
24/24 [==============================] - 15s 632ms/step - loss: 0.7833 - val_lo

In [55]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "functional_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, None, 39)]        0         
_________________________________________________________________
lstm_8 (LSTM)                [(None, 256), (None, 256) 303104    
Total params: 303,104
Trainable params: 303,104
Non-trainable params: 0
_________________________________________________________________


In [56]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [57]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()



Model: "functional_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, None, 53)]   0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, None, 256),  317440      input_10[0][0]                   
                                                                 input_11[0][0]       

In [58]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [59]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [60]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: run 
정답 문장:  cours   
번역기가 번역한 문장:  ne sous pas pas  
-----------------------------------
입력 문장: i left 
정답 문장:  je suis partie  
번역기가 번역한 문장:  je ne suis pas de mon  
-----------------------------------
입력 문장: call us 
정답 문장:  appelez nous   
번역기가 번역한 문장:  nous avons pas  
-----------------------------------
입력 문장: how nice 
정답 문장:  comme c est gentil   
번역기가 번역한 문장:  tom est pas de mon  
-----------------------------------
입력 문장: turn left 
정답 문장:  tourne à gauche  
번역기가 번역한 문장:  qui le t a pas  


# 구글 번역기로 확인한 결과 번역이 정확하다는것을 알수 있었습니다.